# Collecting timelines of the different accounts

## Set up notebook

In [11]:
import pandas as pd
import math
#import PyDown
import tweepy
import jsonpickle
from collections import defaultdict
import re
from pathlib import Path

In [23]:
# Set paths
_nb = Path(Path.cwd()) # Path to notebooks
_dp = _nb.parent / 'Digital Methods' # Path to data

## Set up access to Twitter API through Tweepy

In [14]:
# Consumer:
CONSUMER_KEY    = 'SUxVgkJyHx1h9CSWYQ1n7ZPRB'
CONSUMER_SECRET = 'QAnvbqILy5etrrysaAyVDTf5UOoYhbZK1twEmty60RY1MiiQI8'

# Access:
ACCESS_TOKEN  = '1113067170903154691-qhfOUOLz1uonmKxHOOftR2tdSAPdMd'
ACCESS_SECRET = 'zY0WN72WlxTf5Mtfr3JmHtvYZD6GEgv6v40iNoqe9WXkB'

# Setting up the authentication
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# Defining the API
api = tweepy.API(auth, 
          wait_on_rate_limit=True) #wait_on_rate_limit_notify=True)

## Requests tweets from API

In [15]:
# Twitter handles for Danish NGOs
d = {'Ursula von der Leyen': 'vonderleyen'
    }

In [16]:
# Request tweets for each NGO
fname = 'tweets' #Filename
with open(fname+'.json', 'w') as f:
    print('Scraping timelines of {} user(s)'.format(len(fname)))
    for user in d.values():
        tweetCount = 0
        # Scraping status objects from the 'user' screen name
        for status in tweepy.Cursor(api.user_timeline, 
                                    screen_name=user, 
                                    tweet_mode="extended"
                                   ).items(): #Add an integeter in the items.() paranthesis if you want to specify the number of tweets collected: eg `items(100)´ for the 100 most recent tweets
            f.write(jsonpickle.encode(status._json, unpicklable=False) + '\n')
            tweetCount += 1
        print('Done collecting {} tweets for user {}'.format(tweetCount, user))

Scraping timelines of 6 user(s)
Done collecting 3218 tweets for user vonderleyen


# Parse

In [31]:
def tweets_to_df(path):
    '''
    Creating a pandas DataFrame object from the json files collected through the Twitter API.
    Has to be looped through each file.
    '''

    # Creating a list from the json-file with each list element as a dictionary-entry
    tweets = list(open(path, 'rt'))

    # Craeting empty lists for storing the information from the dictionaries
    # Tweet info
    text = []               # The tweets string-text. Retweets are shortened
    post_id = []            # The tweets unique ID
    weekday = []            # The weekday the tweet was posted
    month = []              # The month the tweet was posted
    day = []                # The calendar-day the tweets was posted
    timestamp = []          # The timestamp for the tweet
    datetime = []           # Datatime format
    lang = []               # Language
    ## Tweet content
    hashtags = []           # List of hashtags
    links = []              # List of links
    mentions_names = []     # List of mentioned screen names
    mentions_ids = []       # List of mentioned ID's

    favorites = []          # Number of favorites the tweet had received at the time of scraping
    retweets = []           # Number of retweets the tweet had received at the time of scraping
    replies = []            # Number of replies the tweet had received at the time of scraping
    quotes = []             # Number of quotes the tweet had received at the time of scraping

    ### Tweet meta-data
    source = []             # The device-source for the tweet. Not cleaned yet
    post_link = []          # A link to the post
    coordinates = []        # Geo-data for the post
    place = []              # Another type of geo-data for the post in dictionary format

    ## Retweet
    retweet = []            # Binary retweet classifier (1 if it's a retweet, 0 if it's not)
    retweet_id = []         # The retweeted tweets unique ID
    retweeted_user = []     # The retweeted user's username
    retweeted_user_id = []  # The retweeted user's ID

    # Quote
    quote = []              # Binary quote classifier (1 if it's a quote, 0 if it's not)
    quote_id = []           # The quoted tweets unique ID
    quoted_user = []        # The quoted user's username
    quoted_user_id = []     # The quoted user's ID

    # # Reply
    reply = []              # Binary reply classifier (1 if it's a reply, 0 if it's not)
    reply_id = []           # The unique ID of the tweet replied to
    reply_user = []         # The username of the user replied to
    reply_user_id =  []     # The ID of the user replied to

    # # User 'meta-info       # All info collected at the time of SCRAPING (this varies!!)
    screen_name = []        # The screen name (eg: @'screen_name')
    user = []               # The name displayed
    user_id = []            # The user's ID
    bio = []                # The bio-text
    followers = []          # The number of followers
    following = []          # The number of people the user follows
    listed_count = []       # The number of times the user has been listed
    location = []           # The self-registered location of the user (not the tweet)
    verified =  []          # If the user is Veried (boolean)
    
    # Query
    query = []              # Query number
    query_v = []            # Query version
    query_d = []            # Query date
    
    for l in tweets:
        data = jsonpickle.decode(l)

        if 'retweeted_status' in data:
            if 'extended_tweet' in data['retweeted_status']:
                text.append(data['retweeted_status']['extended_tweet']['full_text'])
            else:
                text.append(data['retweeted_status']['full_text'])
        else:
            if 'extended_tweet' in data:
                text.append(data['extended_tweet']['full_text'])
            else:
                text.append(data['full_text'])
    
        if 'extended_tweet' in data:
            hashtags.append([x['text'] for x in data['extended_tweet']['entities']['hashtags']])

            mentions_names.append([x['screen_name'] for x in data['extended_tweet']['entities']['user_mentions']])

            mentions_ids.append([x['id_str'] for x in data['extended_tweet']['entities']['user_mentions']])

            links.append([x['expanded_url'] for x in data['extended_tweet']['entities']['urls']])
        
        else:
            hashtags.append([x['text'] for x in data['entities']['hashtags']])

            mentions_names.append([x['screen_name'] for x in data['entities']['user_mentions']])

            mentions_ids.append([x['id_str'] for x in data['entities']['user_mentions']])

            links.append([x['expanded_url'] for x in data['entities']['urls']])

        # Post ID
        post_id.append(data['id_str'])

        # Decompose date
        date = data['created_at']
        weekday.append(date.split(' ')[0])
        month.append(date.split(' ')[1])
        day.append(date.split(' ')[2])

        # Time
        timestamp.append(date.split(' ')[3])

        # Datetime
        datetime.append(date)
        
        # Language
        lang.append(data['lang'])

        # Favorites count
        favorites.append(data['favorite_count'])

        # Retweets count
        retweets.append(data['retweet_count'])        

        # Replies count
        try:
            replies.append(data['reply_count'])   
        except KeyError:
            replies.append(0)
        
        # Quotes count
        try:
            quotes.append(data['quote_count'])   
        except KeyError:
            quotes.append(0)
        
        # Type of source
        source.append(data['source'])

        # Link to the post
        post_link.append('https://twitter.com/' + data['user']['screen_name'] + '/status/' + data['id_str'])

        # Geo-data
        coordinates.append(data['coordinates'])
        place.append(data['place'])
        
        # Is retweet
        if 'retweeted_status' not in data:
            retweet.append(0)
            retweet_id.append(None)
            retweeted_user.append(None)
            retweeted_user_id.append(None)
        else:
            retweet.append(1)
            retweet_id.append(data['retweeted_status']['id_str'])
            retweeted_user.append(data['retweeted_status']['user']['screen_name'])
            retweeted_user_id.append(data['retweeted_status']['user']['id_str'])

        # Is quote
        if 'quoted_status' not in data:
            quote.append(0)
            quote_id.append(None)
            quoted_user.append(None)
            quoted_user_id.append(None)
        else:
            quote.append(1)
            quote_id.append(data['quoted_status']['id_str'])
            quoted_user.append(data['quoted_status']['user']['screen_name'])
            quoted_user_id.append(data['quoted_status']['user']['id_str'])

        # Is reply
        if data['in_reply_to_status_id'] == None:
            reply.append(0)
            reply_id.append(None)
            reply_user.append(None)
            reply_user_id.append(None)
        else:
            reply.append(1)
            reply_id.append(data['in_reply_to_status_id_str'])
            reply_user.append(data['in_reply_to_screen_name'])
            reply_user_id.append(data['in_reply_to_user_id_str'])

        # Add screen name
        screen_name.append(data['user']['screen_name'])

        # Add user
        user.append(data['user']['name'])

        # User ID
        user_id.append(data['user']['id_str'])

        # User bio
        bio.append(data['user']['description'])

        # Followers number
        followers.append(data['user']['followers_count'])

        # Following number
        following.append(data['user']['friends_count'])

        # Listed count
        listed_count.append(data['user']['listed_count'])

        # User location
        location.append(data['user']['location'])

        # Verified
        verified.append(data['user']['verified'])

    # Create a dictionary from the lists
    d = {'text': text,
         'post_id' : post_id,
         'weekday': weekday,
         'month' : month,
         'day': day,
         'timestamp' : timestamp,
         'datetime' : datetime,
         'lang' : lang,

         'hashtags' : hashtags,
         'links' : links,
         'mentions_names' : mentions_names,
         'mentions_ids' : mentions_ids,
         
         'favs': favorites,
         'retweets': retweets,
         'replies': replies,
         'quotes' : quotes,
         
         'source' : source,
         'post_link' : post_link,
         'coordinates' : coordinates,
         'place' : place,

         'is_retweet' : retweet,
         'retweet_id' : retweet_id,
         'retweeted_user' : retweeted_user,
         'retweeted_user_id' : retweeted_user_id,

         'is_quote' : quote,
         'quote_id' : quote_id,
         'quoted_user' : quoted_user,
         'quoted_user_id' : quoted_user_id,

         'is_reply': reply,
         'reply_id' : reply_id,
         'reply_to_user': reply_user,
         'reply_to_user_id' : reply_user_id,

         'screen_name' : screen_name,
         'user' : user,
         'user_id' : user_id,
         'bio' : bio,
         'followers': followers,
         'following' : following,
         'listed_count' : listed_count,
         'location' : location,
         'verified' : verified,
          }

    # Create the DataFrame from the dictionary
    return pd.DataFrame(data = d)

In [39]:
#Run the function
tweets_to_df('tweets.json')

,text,post_id,weekday,month,day,timestamp,datetime,lang,hashtags,links,...,reply_to_user_id,screen_name,user,user_id,bio,followers,following,listed_count,location,verified
0,Glad to announce a further €200 million in EU ...,1522165138983378946,Thu,May,05,10:43:44,Thu May 05 10:43:44 +0000 2022,en,[],[https://twitter.com/i/broadcasts/1eaKbNVRREdKX],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169252,209,5655,,True
1,Today I’m in Warsaw for the International Dono...,1522150637978411008,Thu,May,05,09:46:06,Thu May 05 09:46:06 +0000 2022,en,[],[],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169252,209,5655,,True
2,The recovery package for Ukraine will bring th...,1521785907745021953,Wed,May,04,09:36:48,Wed May 04 09:36:48 +0000 2022,en,[],[],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169252,209,5655,,True
3,We want Ukraine to win this war.\n\nYet so muc...,1521745710068248577,Wed,May,04,06:57:04,Wed May 04 06:57:04 +0000 2022,en,[],[],...,1146329871418843136,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169252,209,5655,,True
4,"Finally, we now propose a ban on Russian oil. ...",1521744714403307520,Wed,May,04,06:53:07,Wed May 04 06:53:07 +0000 2022,en,[],[],...,1146329871418843136,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169252,209,5655,,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,Angekommen in Brüssel und direkt zum ersten Tr...,1146760506356748288,Thu,Jul,04,12:39:45,Thu Jul 04 12:39:45 +0000 2019,de,[],[],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169253,209,5655,,True
3214,À peine arrivée à Bruxelles la première rencon...,1146760423183720448,Thu,Jul,04,12:39:25,Thu Jul 04 12:39:25 +0000 2019,fr,[],[],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169253,209,5655,,True
3215,In Brussels and straight off to the first meet...,1146760351830171648,Thu,Jul,04,12:39:08,Thu Jul 04 12:39:08 +0000 2019,en,[],[],...,None,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169253,209,5655,,True
3216,"@Pertsch @Izyy Ja, hier twittere ich selbst! M...",1146444658618314752,Wed,Jul,03,15:44:41,Wed Jul 03 15:44:41 +0000 2019,de,[],[],...,68662384,vonderleyen,Ursula von der Leyen,1146329871418843136,President of the @EU_Commission. Mother of sev...,1169253,209,5655,,True


In [56]:
df = pd.read_json('tweets.json', lines=True)
df.to_csv('data.csv')

In [58]:
df_ = pd.read_csv('data.csv')
df_

,Unnamed: 0,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,...,favorited,retweeted,possibly_sensitive,lang,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status_permalink,quoted_status,retweeted_status
0,0,2022-05-05 10:43:44+00:00,1522165138983378946,1522165138983378944,Glad to announce a further €200 million in EU ...,False,"[0, 221]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://studio.twitter.com"" rel=""nofo...",NaN,...,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-05-05 09:46:06+00:00,1522150637978411008,1522150637978411008,Today I’m in Warsaw for the International Dono...,False,"[0, 273]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,0.0,en,"{'media': [{'id': 1522150631724789760, 'id_str...",NaN,NaN,NaN,NaN,NaN
2,2,2022-05-04 09:36:48+00:00,1521785907745021953,1521785907745021952,The recovery package for Ukraine will bring th...,False,"[0, 241]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://studio.twitter.com"" rel=""nofo...",NaN,...,False,False,0.0,en,"{'media': [{'id': 1521785495042334720, 'id_str...",NaN,NaN,NaN,NaN,NaN
3,3,2022-05-04 06:57:04+00:00,1521745710068248577,1521745710068248576,We want Ukraine to win this war.\n\nYet so muc...,False,"[0, 261]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.521745e+18,...,False,False,0.0,en,"{'media': [{'id': 1521745644716802049, 'id_str...",NaN,NaN,NaN,NaN,NaN
4,4,2022-05-04 06:53:07+00:00,1521744714403307520,1521744714403307520,"Finally, we now propose a ban on Russian oil. ...",False,"[0, 268]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.521744e+18,...,False,False,0.0,en,"{'media': [{'id': 1521744593322188802, 'id_str...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,3213,2019-07-04 12:39:45+00:00,1146760506356748288,1146760506356748288,Angekommen in Brüssel und direkt zum ersten Tr...,False,"[0, 207]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,False,False,0.0,de,"{'media': [{'id': 1146759505239302144, 'id_str...",NaN,NaN,NaN,NaN,NaN
3214,3214,2019-07-04 12:39:25+00:00,1146760423183720448,1146760423183720448,À peine arrivée à Bruxelles la première rencon...,False,"[0, 229]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,False,False,0.0,fr,"{'media': [{'id': 1146759580417974272, 'id_str...",NaN,NaN,NaN,NaN,NaN
3215,3215,2019-07-04 12:39:08+00:00,1146760351830171648,1146760351830171648,In Brussels and straight off to the first meet...,False,"[0, 207]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,...,False,False,0.0,en,"{'media': [{'id': 1146760066080616449, 'id_str...",NaN,NaN,NaN,NaN,NaN
3216,3216,2019-07-03 15:44:41+00:00,1146444658618314752,1146444658618314752,"@Pertsch @Izyy Ja, hier twittere ich selbst! M...",False,"[15, 93]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1.146410e+18,...,False,False,NaN,de,NaN,NaN,NaN,NaN,NaN,NaN


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5d4bb95b-788c-4897-b34f-02e3344e21fe' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>